In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.warp import reproject, calculate_default_transform, Resampling
from rasterio.features import shapes
from rasterio.features import rasterize
from rasterio.transform import from_origin
import rasterio.mask
from rasterio.io import MemoryFile
from shapely.geometry import shape
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:

nc_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/wind_speed.nc"
uk_boundary_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk boundary.geojson"

ds = xr.open_dataset(nc_path)
da = ds["wind_speed"]  

wind_layer = da.mean(dim="season", skipna=True)

uk = gpd.read_file(uk_boundary_path).to_crs(target_crs)


vmin, vmax = 3.0, 7.0

fig, ax = plt.subplots(figsize=(9, 11), dpi=900)  

mesh = ax.pcolormesh(
    ds["x"].values,
    ds["y"].values,
    wind_layer.values,
    shading="auto",
    vmin=vmin,
    vmax=vmax
)

uk.boundary.plot(ax=ax, color="black", linewidth=0.8, zorder=5)

cbar = plt.colorbar(mesh, ax=ax, fraction=0.035, pad=0.02, extend="both")
cbar.set_label("Wind speed (m/s)")

ax.set_aspect("equal")
ax.set_axis_off()
plt.tight_layout()

out_png = "/Users/lct/Desktop/wind_speed_3to7_highres.png"
plt.savefig(out_png, dpi=900, bbox_inches="tight")
plt.show()


In [ ]:

threshold = 4.5

ds = xr.open_dataset(nc_path)
da = ds["wind_speed"]
wind_layer = da.mean(dim="season", skipna=True)

uk = gpd.read_file(uk_boundary_path).to_crs(target_crs)

mask = np.where(np.isfinite(wind_layer.values), (wind_layer.values > threshold).astype(np.uint8), np.nan)
mask = np.ma.masked_invalid(mask)

cmap = ListedColormap(["purple", "green"])
cmap.set_bad(alpha=0)

fig, ax = plt.subplots(figsize=(9, 11), dpi=900)

mesh = ax.pcolormesh(
    ds["x"].values,
    ds["y"].values,
    mask,
    shading="auto",
    cmap=cmap,
    vmin=0,
    vmax=1
)

uk.boundary.plot(ax=ax, color="black", linewidth=0.8, zorder=5)

ax.legend(
    handles=[
        Patch(facecolor="green", edgecolor="none", label=f"> {threshold} m/s"),
        Patch(facecolor="purple", edgecolor="none", label=f"≤ {threshold} m/s"),
    ],
    loc="upper left",
    frameon=True
)

ax.set_aspect("equal")
ax.set_axis_off()
plt.tight_layout()

out_png = "/Users/lct/Desktop/wind_speed_gt{threshold}_binary.png"
plt.savefig(out_png, dpi=900, bbox_inches="tight")
plt.show()